# QA tests on FiberFlat

## Load 'true' model spectrum

In [20]:
# imports
import specter.throughput as spec_thru
import specter.psf.spotgrid as SpotGridPSF
sys.path.append(os.path.abspath("/Users/xavier/DESI/desisim_v0.4.1/desisim/"))
import interpolation as desi_interp
import io as desisim_io

In [3]:
# Read flat
infile = '/Users/xavier/DESI/CALIBS/flat-3100K-quartz-iodine.fits'
hdu = fits.open(infile)
hdu.info()

Filename: /Users/xavier/DESI/CALIBS/flat-3100K-quartz-iodine.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      11   (67461,)     float64   


In [9]:
flux = hdu[0].data
hdr = hdu[0].header
# from desisim_io
wave = hdr['CRVAL1'] + np.arange(hdr['NAXIS1'])*hdr['CDELT1']
if hdr['LOGLAM'] == 1:
    wave = 10**wave

In [10]:
#- resample to 0.2 A grid
dw = 0.2
ww = np.arange(wave[0], wave[-1]+dw/2, dw)
flux = desi_interp.resample_flux(ww, wave, flux)

In [12]:
xdb.xplot(ww,flux)

## Apply thruput

In [13]:
thru = spec_thru.load_throughput('/Users/xavier/DESI/desimodel/data/throughput/thru-b.fits')

In [22]:
# Load PSF
psf_fil = '/Users/xavier/DESI/desimodel/data/specpsf/psf-b.fits'
psf = SpotGridPSF.SpotGridPSF(psf_fil)

In [40]:
# Generate photons from thruput
ii = (psf.wmin <= ww) & (ww <= psf.wmax)
phot = thru.photons(ww[ii], flux[ii], units=hdr['BUNIT'], objtype='CALIB')

In [41]:
# Plot
xdb.xplot(ww[ii],phot)

## Write to file

In [35]:
from linetools.spectra.xspectrum1d import XSpectrum1D

In [36]:
xspec = XSpectrum1D.from_tuple((ww[ii],phot))

In [37]:
xspec.write_to_fits('/Users/xavier/DESI/CALIBS/b-flat-phot_spec.fits')

Wrote spectrum to /Users/xavier/DESI/CALIBS/b-flat-phot_spec.fits
